# Tweet Turing Test: Detecting Disinformation on Twitter  

|          | Group #2 - Disinformation Detectors                     |
|---------:|---------------------------------------------------------|
| Members  | John Johnson, Katy Matulay, Justin Minnion, Jared Rubin |
| Notebook | `02_preprocess.ipynb`                                   |
| Purpose  | Apply a pre-processing pipeline to merged data.         |

> (TODO - write more explaining notebook)

# 1 - Setup

In [1]:
# imports from Python standard library
import json
import logging
import os

# imports requiring installation
#   connection to Google Cloud Storage
from google.cloud import storage            # pip install google-cloud-storage
from google.oauth2 import service_account   # pip install google-auth

#  data science packages
import demoji                               # pip install demoji
import numpy as np                          # pip install numpy
import pandas as pd                         # pip install pandas

In [2]:
# imports from tweet_turing.py
from tweet_turing import get_json_files, load_local_json, get_gcp_storage_client, get_gcp_bucket, \
    list_gcp_objects, get_gcp_object_as_json, get_gcp_object_as_text, set_gcp_object_from_json, \
    is_retweet, is_retweet_alt

# imports from tweet_turing_paths.py
from tweet_turing_paths import local_data_paths, local_snapshot_paths, gcp_data_paths, \
    gcp_snapshot_paths, gcp_project_name, gcp_bucket_name, gcp_key_file

## Local or Cloud?

Decide here whether to run notebook with local data or GCP bucket data
 - if the working directory of this notebook has a "../data/" folder with data loaded (e.g. working on local computer or have data files loaded to a cloud VM) then use the "local files" option and comment out the "gcp bucket files" option
 - if this notebook is being run from a GCP VM (preferrably in the `us-central1` location) then use the "gcp bucket files" option and comment out the "local files" option

In [3]:
# option: local files
local_or_cloud: str = "local"   # comment/uncomment this line or next

# option: gcp bucket files
#local_or_cloud: str = "cloud"   # comment/uncomment this line or previous

# don't comment/uncomment for remainder of cell
if (local_or_cloud == "local"):
    data_paths = local_data_paths
    snapshot_paths = local_snapshot_paths
elif (local_or_cloud == "cloud"):
    data_paths = gcp_data_paths
    snapshot_paths = gcp_snapshot_paths
else:
    raise ValueError("Variable 'local_or_cloud' can only take on one of two values, 'local' or 'cloud'.")
    # subsequent cells will not do this final "else" check

In [4]:
# this cell only needs to run its code if local_or_cloud=="cloud"
#   (though it is harmless if run when local_or_cloud=="local")
gcp_storage_client: storage.Client = None
gcp_bucket: storage.Bucket = None

if (local_or_cloud == "cloud"):
    gcp_storage_client = get_gcp_storage_client(project_name=gcp_project_name, key_file=gcp_key_file)
    gcp_bucket = get_gcp_bucket(storage_client=gcp_storage_client, bucket_name=gcp_bucket_name)

# 2 - Troll Tweets (CSV) Pre-processing

## 2.1 Load CSV Snapshot (from prior merge step)

In [5]:
# load the merged troll tweet CSV snapshot file
csv_filename: str = "csv_snapshot.csv"
csv_path: str = f"{snapshot_paths['csv_snapshot']}{csv_filename}"
troll_df_raw: pd.DataFrame = pd.DataFrame()

if (local_or_cloud == "local"):
    troll_df_raw = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
elif (local_or_cloud == "cloud"):
    pass

## 2.2 Filter for *"Only English language tweets"*

In [6]:
# filter for english language tweets only
#   - relevant dataframe column is `language`
mask_lang_en: pd.Series = (troll_df_raw['language'] == 'English')

troll_df = troll_df_raw[mask_lang_en]

## 2.3 Extract columns of interest

In [7]:
# extract only the columns we will use for later steps
cols_to_keep = [
    'external_author_id',
    'author',
    'content',
    'region',
    'language',
    'publish_date',
    'following',
    'followers',
    'updates',
    'retweet',
    'account_category',
    'tweet_id',
    'tco1_step1'
    ]

troll_df = troll_df[cols_to_keep]

print("Troll Dataframe Shape (rows, cols):", troll_df.shape)

Troll Dataframe Shape (rows, cols): (2116867, 13)


## 2.4 Derive new feature: `data_source`

This feature is setup as a constant value __"Troll"__ for this subset of the dataset to indicate that the data originates from the troll tweets CSV snapshot file. The tweets obtained from Twitter API (in JSON files) have the same feature added by the `01_merge` notebook, but their values are either __"verified_user"__ or __"verified_random"__.

In [8]:
troll_df['data_source'] = 'Troll'

## 2.5 Align column names

In [9]:
# setup rename mapping
#   key = old column name; value = new column name
col_name_mapping = {
    "retweet": "is_retweet",
    "tco1_step1": "full_url",
}

troll_df.rename(columns=col_name_mapping, inplace=True)

# 3 - Authentic Tweets (JSON) Pre-processing

## 3.1 Load JSON Snapshot (from prior merge step)

In [10]:
# load the merged troll tweet CSV snapshot file
json_filename: str = "json_snapshot.json"
json_path: str = f"{snapshot_paths['json_snapshot']}{json_filename}"

json_data: list = []

if (local_or_cloud == "local"):
    json_data = load_local_json(json_path)
elif (local_or_cloud == "cloud"):
    json_data = load_gcp_json(gcp_bucket, json_path)

## 3.2 Transform from JSON to tabular form

Apply the pandas function `json_normalize()` to flatten JSON dict

In [11]:
# convert json to pandas dataframe using normalize to flatten dict
authentic_df_raw = pd.json_normalize(json_data)

## 3.X Transform: Encode as 'utf-8'

Pipeline step skipped, data is already utf-8. Noting here so pipeline diagram can be updated, then delete this cell.

## 3.3 Extract columns of interest (1 of 2)

We'll first extract columns into a copy of the `authentic_df_raw` dataframe. This copy will fork off into a stand-alone snapshot file for use with EDA of the authentic tweets. Before forking off, though, we'll add in our derived features.

Later we'll modify the `authentic_df_raw` dataframe to keep only the columns we intend to merge with the `troll_df` dataframe.

In [12]:
# fork off with a larger subset of columns for later use in authentic-tweet-specific EDA
cols_keep_EDA = [
    'author_id',
    'created_at',
    'id',
    'text',
    'lang',
    'referenced_tweets',
    'public_metrics.retweet_count', 
    'public_metrics.reply_count', 
    'public_metrics.like_count', 
    'public_metrics.quote_count',
    'author.location', 
    'author.name', 
    'author.username', 
    'author.public_metrics.followers_count',
    'author.public_metrics.following_count', 
    'author.entities.url.urls', 
    'author.created_at',
    'author.verified', 
    'context_annotations', 
    'entities.annotations', 
    'entities.mentions',
    'entities.hashtags', 
    'entities.urls',
    'data_source'
    ]

# setup a new dataframe with subset of columns
authentic_df_eda = authentic_df_raw[cols_keep_EDA].copy()

## 3.4 Derive new features

### 3.4.1 Derive new feature: `is_retweet`

Two ways to identify a retweeted tweet:  
1. Field `text` starts with the string "`RT @`", though we need to determine if this can be faked.
2. Field `referenced_tweets` meets all the following criteria:
    - is not `NaN`
    - contains a list with at least one element
    - the first (index=0) element is a dict where key "`type`" has value "`retweeted`"

In [14]:
# derive feature `is_retweet`
#   method 1
new_column = authentic_df_eda.apply(is_retweet, axis='columns')
authentic_df_eda.loc[:, 'is_retweet'] = new_column

In [15]:
# derive feature `is_retweet_alt`
#   method 2
#    1- make a mask of non-NaN `referenced_tweets` rows
notna_mask = authentic_df_eda['referenced_tweets'].notna()

#    2- mask off for non-NaN and apply `is_retweet_alt`, outputting 1 or 0 to masked rows
new_column = authentic_df_eda.loc[notna_mask].apply(is_retweet_alt, axis='columns')
authentic_df_eda.loc[notna_mask, 'is_retweet_alt'] = new_column

#   3- fill in NaN values for any rows filtered out of prior step
authentic_df_eda.loc[~notna_mask, 'is_retweet_alt'] = 0

### 3.4.2 Derive new feature: `updates`

This feature is derived by adding together four public metrics for a given tweet. This matches up with the feature definition from the troll dataset.

In [16]:
update_cols = [
    'public_metrics.retweet_count',
    'public_metrics.reply_count', 
    'public_metrics.like_count',
    'public_metrics.quote_count'
    ]

authentic_df_eda['updates'] = authentic_df_eda[update_cols].sum(axis='columns')

### 3.4.3 Derive new feature: `account_category`

In [17]:
account_category_mapping = {
    True: "Verified_User",
    False: "Unknown",
    }

authentic_df_eda['account_category'] = authentic_df_eda['author.verified'].apply(lambda b: account_category_mapping[b])

## 3.5 Align column names 

In [18]:
# setup rename mapping
#   key = old column name; value = new column name
col_name_mapping = {
    "author_id": "external_author_id", 
    "created_at": "publish_date", 
    "text": "content",
    "lang": "language", 
    "author.location":"region", 
    "author.username":"author",
    "author.name":"full_name",
    "author.public_metrics.followers_count": "followers",
    "author.public_metrics.following_count": "following",
    "id": "tweet_id",
    "entities.urls":"full_url"
    }

authentic_df_eda.rename(columns=col_name_mapping, inplace=True)

## 3.6 Save EDA Snapshot

As a follow-up to section 3.3 above, save a snapshot of the dataset intended for authentic-specific EDA. This dataset will have additional columns beyond the troll dataset.

In [19]:
# note this cell requires package `pyarrow` to be installed in environment
# save `authentic_df_eda` snapshot
parq_filename: str = "authentic_df_eda.parquet.snappy"
parq_path: str = f"{snapshot_paths['json_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    authentic_df_eda.to_parquet(parq_path, engine='pyarrow')
elif (local_or_cloud == "cloud"):
    pass

## 3.7 Drop columns not needed for merge

In [20]:
cols_to_drop = [
    'public_metrics.retweet_count',
    'public_metrics.reply_count', 
    'public_metrics.like_count',
    'public_metrics.quote_count',
    'author.entities.url.urls', 
    'author.created_at', 
    'author.verified',
    'context_annotations', 
    'entities.annotations', 
    'entities.mentions',
    'entities.hashtags',
    'full_name',
    'referenced_tweets',
    ]

authentic_df = authentic_df_eda.drop(columns=cols_to_drop)

In [21]:
# debug
print(len(troll_df.columns))
print(len(authentic_df.columns))

14
15


In [22]:
# debug
sorted(troll_df.columns.to_list())

['account_category',
 'author',
 'content',
 'data_source',
 'external_author_id',
 'followers',
 'following',
 'full_url',
 'is_retweet',
 'language',
 'publish_date',
 'region',
 'tweet_id',
 'updates']

In [23]:
# debug
sorted(authentic_df.columns.to_list())

['account_category',
 'author',
 'content',
 'data_source',
 'external_author_id',
 'followers',
 'following',
 'full_url',
 'is_retweet',
 'is_retweet_alt',
 'language',
 'publish_date',
 'region',
 'tweet_id',
 'updates']

# 4 - Merge (Partially) Pre-processed Tweets

At this stage, the two separate datasets can be merged. Additional pre-processing will still be performed but can be applied to the entire dataset.

## 4.1 Merge

In [25]:
merged_df = pd.concat([troll_df, authentic_df], axis='index')

print("Merged dataframe shape:", merged_df.shape)

Merged dataframe shape: (3624895, 15)


## 4.2 Fix dtypes

Two columns need to be explicitly typed as strings in order to be saved to parquet format.

Using `string[python]` but could also try `string[pyarrow]` to compare performance.

In [38]:
merged_df['tweet_id'] = merged_df['tweet_id'].astype("string[python]")
merged_df['full_url'] = merged_df['full_url'].astype("string[python]")

## 4.3 Save Snapshot

In [39]:
# note this cell requires package `pyarrow` to be installed in environment
# save `merged_df` snapshot
parq_filename: str = "merged_df.parquet.snappy"
parq_path: str = f"{snapshot_paths['json_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    merged_df.to_parquet(parq_path, engine='pyarrow')
elif (local_or_cloud == "cloud"):
    pass

# 5 - Merged DF Pre-Processing

Below are preprocessing steps intended for the full, merged dataframe.

## 5.1 (Optional) Load Snapshot of `merged_df`

Optional cell to load snapshot (parquet file) saved during prior step (4.3).

In [30]:
# note this cell requires package `pyarrow` to be installed in environment
parq_filename: str = "merged_df.parquet.snappy"
parq_path: str = f"{snapshot_paths['json_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    merged_df = pd.read_parquet(parq_path, engine='pyarrow')
    # TODO -> confirm index is maintained from before snapshot
elif (local_or_cloud == "cloud"):
    pass

## 5.2 Remove tweets with null `content` field

In [36]:
merged_df.dropna(subset=['content'], inplace=True)

## 5.3 Derive new feature: `has_url`

In [34]:
# define here for testing, then move to tweet_turing.py
def has_url(tweet_series: pd.Series, search_str: str = 'http') -> int:
    if (tweet_series['content'] is not None):
        return int(search_str in tweet_series['content'])
    else:
        return 0
    #
    # 
    # try:
    #     return int(search_str in tweet_series['content'])
    # except:
    #     print(tweet_series)

In [35]:
new_column = merged_df.apply(has_url, axis='columns')
merged_df.loc[:, 'has_url'] = new_column

## 5.4 Emojis

### 5.4.1 Derive new feature: `emoji_text`

In [ ]:
# define here for testing, then move to tweet_turing.py
''' The following converts a text string with emojis into a list of descriptive text strings.
    Duplicate emojis are captured as each emoji converts to 1 text string.'''
def convert_emoji_list(x):
    lst=[]
    estring = ''
    # import demoji
    # import numpy as np
    if x is not np.nan:
        #extract list of text from demoji func
        lst = demoji.findall_list(x)
        if len(lst)<0:
            return np.nan
     
        else:
            return(lst)

In [ ]:
# apply convert_emoji_list

### 5.4.2 Derive new feature: `emoji_count`

In [ ]:
# count emoji list

## 5.5 Convert `publish_date` to `datetime` format

In [ ]:
# convert

# 6 - Save Preprocessed Data

In [ ]:
# note this cell requires package `pyarrow` to be installed in environment
# save `merged_df` snapshot
parq_filename: str = "merged_df_preprocessed.parquet.snappy"
parq_path: str = f"{snapshot_paths['json_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    merged_df.to_parquet(parq_path, engine='pyarrow')
elif (local_or_cloud == "cloud"):
    pass